In [1]:
import datetime as dt
from itertools import product

import pandas as pd
from GoogleEarth import GoogleEarth

In [2]:
start_time = dt.datetime.strptime("20230203210000", "%Y%m%d%H%M%S")
stop_time = dt.datetime.strptime("20230206000000", "%Y%m%d%H%M%S")
end_time = dt.datetime.strptime("20230301000000", "%Y%m%d%H%M%S")

TypeError: strptime() takes no keyword arguments

In [ ]:
ge = GoogleEarth()